In [26]:
import pandas as pd
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行
import re

In [27]:
df = pd.read_parquet('/Users/luoyonggui/Documents/datasets/work/4/diagMedList/part-00000-c3fd0249-2283-4947-a9f7-2eb85261aba2-c000.snappy.parquet')
df1 = pd.read_parquet('/Users/luoyonggui/Documents/datasets/work/4/diagMedList/part-00001-c3fd0249-2283-4947-a9f7-2eb85261aba2-c000.snappy.parquet')
df = pd.concat([df, df1])
df1 = pd.read_parquet('/Users/luoyonggui/Documents/datasets/work/4/diagMedList/part-00002-c3fd0249-2283-4947-a9f7-2eb85261aba2-c000.snappy.parquet')
df = pd.concat([df, df1])
df1 = pd.read_parquet('/Users/luoyonggui/Documents/datasets/work/4/diagMedList/part-00003-c3fd0249-2283-4947-a9f7-2eb85261aba2-c000.snappy.parquet')
df = pd.concat([df, df1])
df1 = pd.read_parquet('/Users/luoyonggui/Documents/datasets/work/4/diagMedList/part-00004-c3fd0249-2283-4947-a9f7-2eb85261aba2-c000.snappy.parquet')
df = pd.concat([df, df1])
df1 = pd.read_parquet('/Users/luoyonggui/Documents/datasets/work/4/diagMedList/part-00005-c3fd0249-2283-4947-a9f7-2eb85261aba2-c000.snappy.parquet')
df = pd.concat([df, df1])
df1 = pd.read_parquet('/Users/luoyonggui/Documents/datasets/work/4/diagMedList/part-00006-c3fd0249-2283-4947-a9f7-2eb85261aba2-c000.snappy.parquet')
df = pd.concat([df, df1])
del df1




In [28]:
df.shape

(6758316, 3)

In [29]:
df.sample(2)

,OUT_HOSP_DIAGNOSISCODE,OUT_HOSP_DIAGNOSISNAME,medlist
335418,E14.900,糖尿病,"[门诊诊查费(一级医院), 二甲双胍, 瑞格列奈, 阿卡波糖]"
651078,N76.800,阴道炎,"[阴道分泌物检查, 阴道分泌物白细胞酯酶检查, 阴道分泌物过氧化氢浓度检查, 阴道分泌物唾液..."


In [30]:
# 诊断编码和名称相同
dft = df[df['OUT_HOSP_DIAGNOSISCODE']==df['OUT_HOSP_DIAGNOSISNAME']]

In [31]:
dft['OUT_HOSP_DIAGNOSISCODE'].unique()

array(['0', '挂号', 'ZX001', 'C50.100', 'C15.500', 'C16.900', 'C94.701',
       'M32.900', 'C50.900', 'C15.900', 'ZX003', 'C34.900', 'G82.200',
       'ZX005', 'K74.607', 'G80.900', 'C79.101', 'Z99.201', 'C03.900',
       'C18.900', 'C20.x00', 'D61.905', 'C56.x00', 'C61.x00', 'N19.x01',
       'C73.x00', 'C53.900', 'ZX007', 'C64.x00', 'M98030/3', '.泌尿系统疾病',
       'F32.100', 'F32.900', 'F32.902', 'F20.900', 'F32.301', 'F20.901',
       '296.101', 'F31.201', 'F41.102', 'F39  01', '000', 'F41.100',
       'F03.x00', 'F41.101', '无', 'G30.901+', 'F25.000', 'F31.900',
       '295.901', 'G30.000', 'F99.x00', 'F30.201', 'F31.800', 'F20.501',
       'G30.900', 'F42.901', 'F06.200', 'F06.201', 'F30.900', 'F32.901',
       'F31.301', 'F03.x01', 'F42.900', 'F20.300', 'F25.901', 'F33.900',
       'F01.900', 'F20.000', 'F06.801', 'F25.900', '586  02', 'F31.100',
       'G41.101', 'I51.900', 'F33.100', 'C07.x00', 'F31.801', 'F31.300',
       'CX0.004', 'F20.800', 'C34.908', 'D39.101', 'GDB06', 'F23.90

In [32]:
df1 = df[df['OUT_HOSP_DIAGNOSISCODE']!=df['OUT_HOSP_DIAGNOSISNAME']]

In [33]:
# 删除诊断为空的样本
df1 = df1[df1['OUT_HOSP_DIAGNOSISNAME'].notnull()]

In [34]:
# 删除诊断为挂号的样本
# 删除诊断为无的样本
# 删除诊断为...的样本
df1 = df1[df1['OUT_HOSP_DIAGNOSISNAME'].isin(['挂号', '无', '0', '检查', '待查', '其它'])==False]

In [37]:
# 删除 诊断含有特殊病号的 样本
df1 = df1[df1['OUT_HOSP_DIAGNOSISNAME'].str.contains('特殊病号')==False]


In [38]:
def t(s):
    s = re.sub(r'（', '(', s)
    s = re.sub(r'）', ')', s)
    s = re.sub(r'，', ',', s)
    s = re.sub(r'\(.*\)', '', s) # 删除括号内的数据
    s = re.sub(r'\[.*\]', '', s) # 删除中括号内的数据
    return [ii.strip() for ii in re.split(r'\^|,', s) if ii.strip()]

df1['OUT_HOSP_DIAGNOSISNAME_n'] = df1['OUT_HOSP_DIAGNOSISNAME'].apply(t)

In [39]:
def t(l):
    r = []
    for s in l :
        s = re.sub(r'（', '(', s)
        s = re.sub(r'）', ')', s)
        s = re.sub(r'\(.*\)', '', s) # 删除括号内的数据
        s = re.sub(r'\[.*\]', '', s) # 删除中括号内的数据
        r.append(s.strip())
    return r

df1['medlist_n'] = df1['medlist'].apply(t)

### code数量远小于name数量！

In [40]:
df1['OUT_HOSP_DIAGNOSISNAME'].unique().shape

(145129,)

In [41]:
df1['OUT_HOSP_DIAGNOSISCODE'].unique().shape

(12767,)

In [42]:
df1.iloc[0, 2]

['普通病房床位费',
 '住院诊查费（二级医院)',
 '静脉输液（住院/含输液器）',
 '自费诊疗及服务项目',
 '静脉输液（住院/含输液器）',
 '氯化钠',
 '非洛地平',
 '莫沙必利',
 '莫沙必利',
 '维生素B6',
 '泮托拉唑',
 '维生素C',
 '复方甘草甜素\r\n（复方甘草酸苷）',
 '葡萄糖氯化钠',
 '氯化钾',
 '维生素B6',
 '泮托拉唑',
 '维生素C',
 '复方甘草甜素\r\n（复方甘草酸苷）',
 '氯化钾',
 '葡萄糖氯化钠',
 '氯化钠',
 '葡萄糖',
 '等级护理（二级医院）',
 '自费诊疗及服务项目',
 '氯化钾',
 '氯化钾',
 '葡萄糖',
 '硝苯地平',
 '等级护理（二级医院）',
 '维生素B6',
 '泮托拉唑',
 '维生素C',
 '自费诊疗及服务项目',
 '葡萄糖氯化钠',
 '复方甘草甜素\r\n（复方甘草酸苷）',
 '氯化钠',
 '氯化钾',
 '氯化钾',
 '葡萄糖',
 '静脉输液（住院/含输液器）',
 '自费诊疗及服务项目',
 '普通病房床位费',
 '住院诊查费（二级医院)',
 '非洛地平',
 '莫沙必利',
 '莫沙必利',
 '莫沙必利',
 '维生素B6',
 '泮托拉唑',
 '维生素C',
 '复方甘草甜素\r\n（复方甘草酸苷）',
 '氯化钾',
 '氯化钾',
 '葡萄糖氯化钠',
 '氯化钠',
 '葡萄糖',
 '等级护理（二级医院）',
 '自费诊疗及服务项目',
 '静脉输液（住院/含输液器）',
 '自费诊疗及服务项目',
 '非洛地平',
 '莫沙必利',
 '普通病房床位费',
 '住院诊查费（二级医院)',
 '等级护理（二级医院）',
 '自费诊疗及服务项目',
 '静脉输液（住院/含输液器）',
 '自费诊疗及服务项目',
 '静脉注射',
 '一次性注射器',
 '一次性真空采血器',
 '一次性真空采血器',
 '七叶神安片',
 '胰酶',
 '非洛地平',
 '莫沙必利',
 '莫沙必利',
 '维生素B6',
 '泮托拉唑',
 '开塞露',
 '氯化钠',
 '维生素C',
 '葡萄糖氯化钠',
 '氯化钠',
 '复方甘草甜素\r\n（复方甘草酸苷）',
 '氯化钾',
 '葡萄

In [43]:
df1.iloc[:, 3]

0                 [胰腺炎]
3                 [胆囊炎]
4                 [普通病]
5                 [普通病]
6                 [普通病]
7                 [普通病]
8         [月经失调, 子宫良性肿]
10                 [耳痛]
11               [皮肤挫伤]
12                [普通病]
13                [普通病]
14                [高血压]
15                [普通病]
16                [普通病]
17                [普通病]
18                [普通病]
19                [足感染]
20              [疱疹性湿疹]
22                [普通病]
23                 [便血]
24                [阴道炎]
25               [支气管炎]
27                [普通病]
28                [普通病]
29                 [普通]
30                [普通病]
31                 [感冒]
32                [高血压]
33                [普通病]
34             [上呼吸道感染]
              ...      
965916             [扭伤]
965917             [跌倒]
965918             [皮炎]
965919           [软组织伤]
965920          [流行性感冒]
965921          [流行性感冒]
965923       [急性上呼吸道感染]
965924           [睡眠障碍]
965925             [胃炎]
965926             [咳嗽]
965927          

In [44]:
df1.sample(10)

,OUT_HOSP_DIAGNOSISCODE,OUT_HOSP_DIAGNOSISNAME,medlist,OUT_HOSP_DIAGNOSISNAME_n,medlist_n
54119,G43.000,普通型,"[颈腰康胶囊, 麝香镇痛膏]",[普通型],"[颈腰康胶囊, 麝香镇痛膏]"
98043,J39.900,上呼吸道疾病,"[黄连上清丸（颗粒、胶囊、片）, 乳康丸（颗粒、胶囊、片）]",[上呼吸道疾病],"[黄连上清丸, 乳康丸]"
725504,I10.x00,高血压病,[阿托伐他汀],[高血压病],[阿托伐他汀]
574331,J31.200,慢性咽炎,[金莲花颗粒（胶囊、片）],[慢性咽炎],[金莲花颗粒]
726671,J42.x00,慢性支气管炎,"[头孢克肟, 莫匹罗星]",[慢性支气管炎],"[头孢克肟, 莫匹罗星]"
800136,R21 01,过敏性皮疹,"[地塞米松, 一般诊疗费（不含输液器/社区服务中心和乡镇卫生院）]",[过敏性皮疹],"[地塞米松, 一般诊疗费]"
356163,J02.905,咽喉痛,[银黄颗粒（胶囊、片）],[咽喉痛],[银黄颗粒]
735225,M10.900,痛风,"[百令胶囊（片）, 美托洛尔, 硝苯地平, 氯沙坦钾]",[痛风],"[百令胶囊, 美托洛尔, 硝苯地平, 氯沙坦钾]"
793613,M25.500,"关节痛,骨质疏松","[碳酸钙D3, 羌活（配方颗粒）, 当归（配方颗粒）, 赤芍（配方颗粒）, 甘草（配方颗粒）...","[关节痛, 骨质疏松]","[碳酸钙D3, 羌活, 当归, 赤芍, 甘草, 一般诊疗费, 大枣, 片姜黄, 黄芪, 防风..."
432917,H10.900,结膜炎,"[一般诊疗费（含输液器/社区服务中心和乡镇卫生院）, 自费中成药, 妥布霉素地塞米松, 头孢呋辛]",[结膜炎],"[一般诊疗费, 自费中成药, 妥布霉素地塞米松, 头孢呋辛]"


In [45]:
def t(arr):
    r = []
    for i in list(arr):
        r.extend(i)
    return r
#     return list(set(r))
dft = df1.groupby('OUT_HOSP_DIAGNOSISNAME')['medlist_n'].agg(t)
dft.head()

OUT_HOSP_DIAGNOSISNAME
"慢性肾炎综合征，局灶节段性肾小球损害\t"                                                [尿常规, 一般诊疗费]
"抗血栓形成药[血小板聚集抑制剂]有害效应\t",高脂血症                                 [氯吡格雷, 阿托伐他汀, 一般诊疗费]
'                                                         [头孢克肟, 小清创, 急诊挂号, 一般诊疗费]
(双)手二度烧伤                         [静脉输液, 左氧氟沙星, 氯化钠, 一次性真空采血器, 一次性真空采血器, 超敏C反应蛋白...
(双)膝关节痛                          [氯测定, 钠测定, 螺旋CT平扫, 省、市离休门诊、住院可使用的一次性材料, 16层及以上...
Name: medlist_n, dtype: object

In [46]:
df2 = df1.drop_duplicates(['OUT_HOSP_DIAGNOSISNAME'])
del df2['medlist_n']
pd.merge(df2, dft, left_on='OUT_HOSP_DIAGNOSISNAME', right_index=True)[['OUT_HOSP_DIAGNOSISNAME_n', 'medlist_n']].to_pickle('data_ready.pkl')

